In [1]:
pip install transformers datasets tqdm konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 10.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from urllib import request
import requests
from bs4 import BeautifulSoup 
from urllib.request import urlopen, urlretrieve
from urllib.parse import quote_plus # 한글 처리용 함수
import pandas as pd
import numpy as np
import time
import os
from konlpy.tag import Okt

In [3]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("klue/bert-base")
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# pipeline 종류 

'audio-classification', 'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-segmentation', 'image-to-text', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY'

# 뉴스에서 타이틀만 가져오기

In [81]:
url_str = 'https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q=%EB%89%B4%EC%8A%A4'
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

titles = []

for i in range(10):
    url = url_str + str(i+1)
    html_page = requests.get(url, header)
    soup = BeautifulSoup(html_page.text, 'lxml') 

    title = soup.select(".tit_main.fn_tit_u")  # select_one 는 한개만 받아옴

    for t in title:
        titles.append(t.text)


# url= 'https://news.daum.net/'
# header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

# titles = []

# html_page = requests.get(url, header)
# soup = BeautifulSoup(html_page.text, 'lxml') 

# title = soup.select(".tit_g")  # select_one 는 한개만 받아옴


# for t in title:
#     titles.append(t.text)

# print(titles)        

# titles

['\n\n                                    대장동 수사팀, ‘이재명 측근 증거인멸 시도’ 의혹 수사 나선다\n                                \n', "\n\n                                    대한항공 본사 리모델링 들어간다…'아시아나 통합 준비'\n                                \n", '\n\n                                    우리사주 물량 폭탄 이겨내니…LG엔솔 공매도도 ‘주춤’\n                                \n', '\n\n                                    "세수 부족 가능성"…금리 결정에 변수로 떠오른 \'추경\' [조미현의 외환·금융 워치]\n                                \n', '\n\n                                    김건희 팬클럽 강신업, 컷오프되자 "4000만원 갈취" 탈당 반발\n                                \n', "\n\n                                    [주식 초고수는 지금] '역대급 실적' 예상되는 SK이노베이션 매수 1위\n                                \n", '\n\n                                    세대별 불만 뚜렷···20대 병역·3040대 교통·50대 이상 부동산\n                                \n', '\n\n                                    "난방비 유독 더 나오는 거 같더니"..오래된 아파트, 이게  원인이었네\n                                \n', '\n\n                                    유흥업소 불법 취업 필리핀 여성 5명

['\n\n                                    대장동 수사팀, ‘이재명 측근 증거인멸 시도’ 의혹 수사 나선다\n                                \n',
 "\n\n                                    대한항공 본사 리모델링 들어간다…'아시아나 통합 준비'\n                                \n",
 '\n\n                                    우리사주 물량 폭탄 이겨내니…LG엔솔 공매도도 ‘주춤’\n                                \n',
 '\n\n                                    "세수 부족 가능성"…금리 결정에 변수로 떠오른 \'추경\' [조미현의 외환·금융 워치]\n                                \n',
 '\n\n                                    김건희 팬클럽 강신업, 컷오프되자 "4000만원 갈취" 탈당 반발\n                                \n',
 "\n\n                                    [주식 초고수는 지금] '역대급 실적' 예상되는 SK이노베이션 매수 1위\n                                \n",
 '\n\n                                    세대별 불만 뚜렷···20대 병역·3040대 교통·50대 이상 부동산\n                                \n',
 '\n\n                                    "난방비 유독 더 나오는 거 같더니"..오래된 아파트, 이게  원인이었네\n                                \n',
 '\n\n                                    유흥업소 불법 취업 필

# 불용어 제거

In [76]:
f = open('korean_stopwords.txt', 'r')  
stopwords = f.read()   
stop_words = set(stopwords.split('\n'))

okt = Okt()

sentence = titles[np.random.randint(len(titles))]
print(sentence)
word_tokens = okt.morphs(sentence)

result = [word for word in word_tokens if not word in stop_words]


sentence =''
for i in range(len(result)):
  sentence += (" " + result[i])

print(result)
print(sentence)

'이자 장사' 4대 금융지주, 작년 순익 16.5조 '역대 최대'
["'", '이자', '장사', "'", '4', '대', '금융', '지주', ',', '작년', '순', '익', '16.5조', "'", '역대', '최대', "'"]
 ' 이자 장사 ' 4 대 금융 지주 , 작년 순 익 16.5조 ' 역대 최대 '


# Consine Similarity zero shot


In [77]:
from torch.nn import functional as F

labels = ["날씨", "정치", "경제", "세계", "스포츠", "생활문화", "IT과학", "사회"]

# sentence = titles[np.random.randint(len(titles))]

# run inputs through model and mean-pool over the sequence
# dimension to get sequence-level representations
inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                     return_tensors='pt',
                                     pad_to_max_length=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
output = model(input_ids, attention_mask=attention_mask)[0]
sentence_rep = output[:1].mean(dim=1)
label_reps = output[1:].mean(dim=1)

# now find the labels with the highest cosine similarities to
# the sentence
similarities = F.cosine_similarity(sentence_rep, label_reps)
closest = similarities.argsort(descending=True)
print(sentence)
for ind in closest:
    print(f'label: {labels[ind]} \t similarity: {similarities[ind]}')

 ' 이자 장사 ' 4 대 금융 지주 , 작년 순 익 16.5조 ' 역대 최대 '
label: 경제 	 similarity: 0.16654343903064728
label: 세계 	 similarity: 0.16310936212539673
label: 정치 	 similarity: 0.16026780009269714
label: 날씨 	 similarity: 0.1515742391347885
label: 스포츠 	 similarity: 0.13451983034610748
label: 생활문화 	 similarity: 0.12947340309619904
label: 사회 	 similarity: 0.1286986768245697
label: IT과학 	 similarity: 0.12604519724845886


#unmasker

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='klue/bert-base')
unmasker("손흥민은 [MASK] 선수 입니다")

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.3288685381412506,
  'token': 4713,
  'token_str': '축구',
  'sequence': '손흥민은 축구 선수 입니다'},
 {'score': 0.043872810900211334,
  'token': 5873,
  'token_str': '엄청난',
  'sequence': '손흥민은 엄청난 선수 입니다'},
 {'score': 0.040990594774484634,
  'token': 11251,
  'token_str': '손흥민',
  'sequence': '손흥민은 손흥민 선수 입니다'},
 {'score': 0.033716849982738495,
  'token': 20482,
  'token_str': '토트넘',
  'sequence': '손흥민은 토트넘 선수 입니다'},
 {'score': 0.030918965116143227,
  'token': 4407,
  'token_str': '독일',
  'sequence': '손흥민은 독일 선수 입니다'}]